In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from keras.layers import LSTM, Dense, SimpleRNN
import matplotlib.pyplot as plt
import statistics
from sklearn.model_selection import train_test_split
from keras.layers import Dense,Dropout,Conv3D,Conv1D,Conv2D,Flatten,MaxPool2D
from sklearn.metrics import accuracy_score
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/prop-5.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0     prop-5      347    1    1    0    1    5     0   0   1  ...  0.000000   
1     prop-5      347    1    1    0    0    5     0   0   0  ...  0.000000   
2     prop-5      347    3    1    0   13   33     3   0  13  ...  0.142857   
3     prop-5      347    1    1    0    0    5     0   0   0  ...  0.000000   
4     prop-5      347    1    1    0    1    5     0   0   1  ...  0.000000   
...      ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
8511  prop-5      362    3    3    0    8   14     1   0   8  ...  0.500000   
8512  prop-5      362    2    3    0   17   17     1   0  17  ...  0.000000   
8513  prop-5      362    2    3    0   12   15     1   0  12  ...  0.000000   
8514  prop-5      362    2    3    0   15   20     1   0  15  ...  0.000000   
8515  prop-5      362   40    1    0    7   48   780   0   7  ...  0.000000   

      moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0       0  0.000000  0.000000   0    0  12.000000       0  0.0000    0  
1       0  0.000000  0.000000   0    0  12.000000       0  0.0000    0  
2       0  0.000000  0.666667   0    0  46.333333       4  1.6667    0  
3       0  0.000000  0.000000   0    0  12.000000       0  0.0000    0  
4       0  0.000000  0.000000   0    0  12.000000       0  0.0000    0  
...   ...       ...       ...  ..  ...        ...     ...     ...  ...  
8511    0  0.947368  0.666667   0    0  13.666667       2  0.6667    0  
8512    0  0.857143  0.625000   1    2  25.000000       1  0.5000    0  
8513    0  0.857143  0.625000   1    1  21.000000       1  0.5000    1  
8514    0  0.857143  0.625000   1    1  41.000000       3  1.5000    0  
8515    0  0.000000  0.616667   0    0   8.425000       3  1.0500    8  

[8516 rows x 23 columns]

In [4]:
ant_master = ant_master.loc[ant_master['version'] == 347]
ant_master

Name  version  wmc  dit  noc  cbo  rfc  lcom  ca  ce  ...       dam  \
0     prop-5      347    1    1    0    1    5     0   0   1  ...  0.000000   
1     prop-5      347    1    1    0    0    5     0   0   0  ...  0.000000   
2     prop-5      347    3    1    0   13   33     3   0  13  ...  0.142857   
3     prop-5      347    1    1    0    0    5     0   0   0  ...  0.000000   
4     prop-5      347    1    1    0    1    5     0   0   1  ...  0.000000   
...      ...      ...  ...  ...  ...  ...  ...   ...  ..  ..  ...       ...   
2866  prop-5      347    4    3    0   11   46     0   0  11  ...  0.200000   
2867  prop-5      347    2    3    0    9   12     1   0   9  ...  0.000000   
2868  prop-5      347    3    3    0    8   14     1   0   8  ...  0.500000   
2869  prop-5      347    2    3    0   29   22     1   0  29  ...  0.000000   
2870  prop-5      347    2    4    0   15   10     1   0  15  ...  0.000000   

      moa       mfa       cam  ic  cbm        amc  max_cc  avg_cc  bug  
0       0  0.000000  0.000000   0    0  12.000000       0  0.0000    0  
1       0  0.000000  0.000000   0    0  12.000000       0  0.0000    0  
2       0  0.000000  0.666667   0    0  46.333333       4  1.6667    0  
3       0  0.000000  0.000000   0    0  12.000000       0  0.0000    0  
4       0  0.000000  0.000000   0    0  12.000000       0  0.0000    0  
...   ...       ...       ...  ..  ...        ...     ...     ...  ...  
2866    0  0.875000  0.833333   2    3  83.000000       6  1.7500    0  
2867    0  0.857143  0.625000   1    1  29.000000       3  1.5000    1  
2868    0  0.947368  0.666667   0    0  13.666667       2  0.6667    0  
2869    0  0.857143  0.625000   1    1  66.000000       6  3.0000    0  
2870    0  0.928571  0.625000   1    1  15.000000       1  0.5000    0  

[2871 rows x 23 columns]

In [5]:
train, test = train_test_split(ant_master, test_size=0.2)

In [6]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [7]:
x_train

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc  dam  moa  \
2292    2    2    0   10   11     1   0  10    2  1.000000   42  0.0    0   
2375    5    5    0   10   15     8   0  10    2  1.000000   52  0.0    0   
2261    2    3    0   12   12     1   0  12    2  1.000000   44  0.0    0   
1094    5    5    0    1   10    10   0   1    5  1.000000   48  0.0    0   
751     6    3    6   11   20    13   6   5    2  0.866667  111  1.0    0   
...   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...  ...  ...   
240     3    1    0   12   18     1   0  12    2  0.900000   91  0.6    0   
1657    4    5    0   14   25     4   1  13    3  1.000000  160  0.0    0   
2120    2    4    0    6    6     1   0   6    2  2.000000   16  0.0    0   
984     9    5    0   18   25     6   6  12    8  0.791667  177  0.0    0   
1571    2    5    0   20   24     1   0  20    2  1.000000  153  0.0    0   

           mfa       cam  ic  cbm        amc  max_cc  avg_cc  
2292  0.800000  0.666667   1    1  19.500000       1  0.5000  
2375  0.875000  0.500000   2    3   8.800000       1  0.6000  
2261  0.857143  0.625000   1    1  20.500000       1  0.5000  
1094  0.846154  1.000000   1    2   8.200000       2  1.2000  
751   0.909091  0.360000   2    3  17.000000       6  2.0000  
...        ...       ...  ..  ...        ...     ...     ...  
240   0.000000  0.666667   0    0  27.666667       4  1.3333  
1657  0.959184  0.555556   2    4  38.250000       1  0.5000  
2120  0.972222  0.666667   2    2   7.000000       1  0.5000  
984   0.870370  0.656250   2    4  18.333333       1  0.7778  
1571  0.937500  0.600000   1    1  75.000000       1  0.5000  

[2296 rows x 20 columns]

In [8]:
x_test

wmc  dit  noc  cbo  rfc  lcom  ca  ce  npm     lcom3  loc  dam  moa  \
2553    3    3    0    8   14     1   0   8    2  0.750000   46  0.5    0   
2353    2    2    0   13   18     0   4   9    2  0.000000   75  1.0    1   
1782    3    6    0    3    8     3   1   2    2  1.166667   29  0.0    0   
111     1    1    0    1    5     0   0   1    0  2.000000   16  0.0    0   
1463    4    5    0   10   13     6   0  10    2  1.000000  103  0.0    0   
...   ...  ...  ...  ...  ...   ...  ..  ..  ...       ...  ...  ...  ...   
115     1    1    0    1    5     0   0   1    0  2.000000   31  0.0    0   
1633    4    5    0   11   18     4   1  10    3  1.000000   67  0.0    0   
1053    2    4    0   22   33     1   0  22    2  1.000000  207  0.0    0   
517     2    4    0   13   13     1   0  13    2  1.000000   72  0.0    0   
2555   11    1    0   46  105    55  13  33    9  1.000000  412  0.0    0   

           mfa       cam  ic  cbm         amc  max_cc  avg_cc  
2553  0.947368  0.666667   0    0   13.666667       2  0.6667  
2353  0.857143  0.666667   0    0   36.000000       1  0.5000  
1782  0.979592  1.000000   3    4    7.666667       1  0.3333  
111   0.000000  0.000000   0    0   12.000000       0  0.0000  
1463  0.918919  0.333333   1    2   23.750000       2  1.0000  
...        ...       ...  ..  ...         ...     ...     ...  
115   0.000000  0.000000   0    0   12.000000       0  0.0000  
1633  0.959184  0.555556   2    4   15.000000       1  0.5000  
1053  0.972222  0.666667   3    3  101.000000       7  3.5000  
517   0.928571  0.625000   1    1   34.000000       1  0.5000  
2555  0.000000  0.136364   0    0   36.181818       3  1.5455  

[575 rows x 20 columns]

In [9]:
y_train = pd.DataFrame(train['bug'])
y_test = pd.DataFrame(test['bug'])

In [10]:
y_train

bug
2292    0
2375    0
2261    0
1094    0
751     0
...   ...
240     0
1657    0
2120    0
984     2
1571    0

[2296 rows x 1 columns]

In [11]:
y_test

bug
2553    0
2353    0
1782    0
111     0
1463    0
...   ...
115     0
1633    0
1053    0
517     0
2555    0

[575 rows x 1 columns]

In [12]:
#Getting the rows and columns size in our data
img_rows, img_cols = 1,20
# Preparing the data for the model.
X_train_matrix = x_train.values
X_test_matrix = x_test.values
Y_train_matrix = y_train.values
Y_test_matrix = y_test.values
# Preparing the data for the model.
Ytrainseries = y_train['bug']
Ytestseries = y_test['bug']

In [13]:
# Preparing the data for the model.
X_train1 = X_train_matrix.reshape(X_train_matrix.shape[0], img_rows, img_cols, 1)
X_test1 = X_test_matrix.reshape(X_test_matrix.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

In [14]:
def define_model():
    # Designing and initializing the model.
    #Building the model
    model = Sequential()
    #add model layers
    model.add(Conv1D(2, kernel_size=2, activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(Dense(1, activation='relu'))
    #compile model using mse as the loss function
    model.compile(loss='mse', optimizer = keras.optimizers.Adam(learning_rate=0.0005), metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()])
    return model

In [15]:
y = y_test['bug'].tolist()
y

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [16]:
total_training_time = []
total_scores = []
total_training_loss_history = []
test_ARE = []
for i in range(30):
    print("Run no: " + str(i))
    model = define_model()
    log_dir = '/home/bavanya/Documents/Training_curves/cnn_propv347/' + 'run' + str(i)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    start = time.time()
    history = model.fit(X_train1, Y_train_matrix, epochs = 100, batch_size = 100, callbacks=[tensorboard_callback])
    stop = time.time()
    total_training_time.append(stop - start) 
    total_training_loss_history.append(history.history)
    score = model.evaluate(X_test1, Y_test_matrix)
    total_scores.append(score) 
    predictions = model.predict(X_test1)
    x = predictions.tolist()
    x = list(np.concatenate(x))
    x = [int(i) for i in x]
    ARE_test = tf.keras.metrics.MeanRelativeError(normalizer=y)
    ARE_test.update_state(y, x)
    test_ARE.append(ARE_test.result().numpy())
    print("score of " + str(i) + "th run is: " + ', '.join(map(str,score)))

Run no: 0
Epoch 1/100
23/23 [==============================] - 0s 2ms/step - loss: 1279.0428 - mse: 1279.0428 - mae: 16.6348 - root_mean_squared_error: 35.7637 - mean_squared_logarithmic_error: 6.2554
Epoch 2/100
23/23 [==============================] - 0s 906us/step - loss: 1128.4388 - mse: 1128.4388 - mae: 15.3317 - root_mean_squared_error: 33.5922 - mean_squared_logarithmic_error: 5.8875
Epoch 3/100
23/23 [==============================] - 0s 1ms/step - loss: 1007.4336 - mse: 1007.4336 - mae: 14.2091 - root_mean_squared_error: 31.7401 - mean_squared_logarithmic_error: 5.5570
Epoch 4/100
23/23 [==============================] - 0s 929us/step - loss: 891.7160 - mse: 891.7160 - mae: 13.2740 - root_mean_squared_error: 29.8616 - mean_squared_logarithmic_error: 5.2709
Epoch 5/100
23/23 [==============================] - 0s 1ms/step - loss: 800.4311 - mse: 800.4311 - mae: 12.3149 - root_mean_squared_error: 28.2919 - mean_squared_logarithmic_error: 4.9641
Epoch 6/100
23/23 [================

23/23 [==============================] - 0s 980us/step - loss: 8.9882 - mse: 8.9882 - mae: 0.1582 - root_mean_squared_error: 2.9980 - mean_squared_logarithmic_error: 0.0589
Epoch 87/100
23/23 [==============================] - 0s 942us/step - loss: 8.6102 - mse: 8.6102 - mae: 0.1560 - root_mean_squared_error: 2.9343 - mean_squared_logarithmic_error: 0.0584
Epoch 88/100
23/23 [==============================] - 0s 916us/step - loss: 8.4638 - mse: 8.4638 - mae: 0.1550 - root_mean_squared_error: 2.9093 - mean_squared_logarithmic_error: 0.0582
Epoch 89/100
23/23 [==============================] - 0s 950us/step - loss: 8.0352 - mse: 8.0352 - mae: 0.1529 - root_mean_squared_error: 2.8346 - mean_squared_logarithmic_error: 0.0580
Epoch 90/100
23/23 [==============================] - 0s 914us/step - loss: 7.7514 - mse: 7.7514 - mae: 0.1517 - root_mean_squared_error: 2.7841 - mean_squared_logarithmic_error: 0.0580
Epoch 91/100
23/23 [==============================] - 0s 928us/step - loss: 7.4820 

23/23 [==============================] - 0s 970us/step - loss: 0.3319 - mse: 0.3319 - mae: 0.0949 - root_mean_squared_error: 0.5761 - mean_squared_logarithmic_error: 0.0525
Epoch 72/100
23/23 [==============================] - 0s 923us/step - loss: 0.3319 - mse: 0.3319 - mae: 0.0949 - root_mean_squared_error: 0.5761 - mean_squared_logarithmic_error: 0.0525
Epoch 73/100
23/23 [==============================] - 0s 965us/step - loss: 0.3319 - mse: 0.3319 - mae: 0.0949 - root_mean_squared_error: 0.5761 - mean_squared_logarithmic_error: 0.0525
Epoch 74/100
23/23 [==============================] - 0s 953us/step - loss: 0.3319 - mse: 0.3319 - mae: 0.0949 - root_mean_squared_error: 0.5761 - mean_squared_logarithmic_error: 0.0525
Epoch 75/100
23/23 [==============================] - 0s 905us/step - loss: 0.3319 - mse: 0.3319 - mae: 0.0949 - root_mean_squared_error: 0.5761 - mean_squared_logarithmic_error: 0.0525
Epoch 76/100
23/23 [==============================] - 0s 945us/step - loss: 0.3319 

23/23 [==============================] - 0s 965us/step - loss: 10.4695 - mse: 10.4695 - mae: 0.2730 - root_mean_squared_error: 3.2357 - mean_squared_logarithmic_error: 0.1093
Epoch 56/100
23/23 [==============================] - 0s 943us/step - loss: 9.6318 - mse: 9.6318 - mae: 0.2584 - root_mean_squared_error: 3.1035 - mean_squared_logarithmic_error: 0.1038
Epoch 57/100
23/23 [==============================] - 0s 884us/step - loss: 8.9064 - mse: 8.9064 - mae: 0.2432 - root_mean_squared_error: 2.9844 - mean_squared_logarithmic_error: 0.0976
Epoch 58/100
23/23 [==============================] - 0s 923us/step - loss: 8.2783 - mse: 8.2783 - mae: 0.2285 - root_mean_squared_error: 2.8772 - mean_squared_logarithmic_error: 0.0913
Epoch 59/100
23/23 [==============================] - 0s 980us/step - loss: 7.4963 - mse: 7.4963 - mae: 0.2181 - root_mean_squared_error: 2.7379 - mean_squared_logarithmic_error: 0.0878
Epoch 60/100
23/23 [==============================] - 0s 954us/step - loss: 6.932

23/23 [==============================] - 0s 932us/step - loss: 0.5641 - mse: 0.5641 - mae: 0.1249 - root_mean_squared_error: 0.7510 - mean_squared_logarithmic_error: 0.0668
Epoch 41/100
23/23 [==============================] - 0s 953us/step - loss: 0.5413 - mse: 0.5413 - mae: 0.1229 - root_mean_squared_error: 0.7357 - mean_squared_logarithmic_error: 0.0659
Epoch 42/100
23/23 [==============================] - 0s 988us/step - loss: 0.5284 - mse: 0.5284 - mae: 0.1212 - root_mean_squared_error: 0.7269 - mean_squared_logarithmic_error: 0.0652
Epoch 43/100
23/23 [==============================] - 0s 931us/step - loss: 0.5131 - mse: 0.5131 - mae: 0.1197 - root_mean_squared_error: 0.7163 - mean_squared_logarithmic_error: 0.0645
Epoch 44/100
23/23 [==============================] - 0s 882us/step - loss: 0.5016 - mse: 0.5016 - mae: 0.1185 - root_mean_squared_error: 0.7083 - mean_squared_logarithmic_error: 0.0640
Epoch 45/100
23/23 [==============================] - 0s 954us/step - loss: 0.4901 

...

23/23 [==============================] - 0s 994us/step - loss: 0.2990 - mse: 0.2990 - mae: 0.1316 - root_mean_squared_error: 0.5468 - mean_squared_logarithmic_error: 0.0479
Epoch 34/100
23/23 [==============================] - 0s 860us/step - loss: 0.2987 - mse: 0.2987 - mae: 0.1315 - root_mean_squared_error: 0.5466 - mean_squared_logarithmic_error: 0.0474
Epoch 35/100
23/23 [==============================] - 0s 827us/step - loss: 0.2991 - mse: 0.2991 - mae: 0.1335 - root_mean_squared_error: 0.5469 - mean_squared_logarithmic_error: 0.0472
Epoch 36/100
23/23 [==============================] - 0s 905us/step - loss: 0.2981 - mse: 0.2981 - mae: 0.1387 - root_mean_squared_error: 0.5460 - mean_squared_logarithmic_error: 0.0474
Epoch 37/100
23/23 [==============================] - 0s 993us/step - loss: 0.2982 - mse: 0.2982 - mae: 0.1436 - root_mean_squared_error: 0.5461 - mean_squared_logarithmic_error: 0.0479
Epoch 38/100
23/23 [==============================] - 0s 862us/step - loss: 0.2982 

23/23 [==============================] - 0s 797us/step - loss: 160.1803 - mse: 160.1803 - mae: 0.4664 - root_mean_squared_error: 12.6562 - mean_squared_logarithmic_error: 0.0872
Epoch 19/100
23/23 [==============================] - 0s 820us/step - loss: 142.9084 - mse: 142.9084 - mae: 0.4314 - root_mean_squared_error: 11.9544 - mean_squared_logarithmic_error: 0.0827
Epoch 20/100
23/23 [==============================] - 0s 871us/step - loss: 128.2813 - mse: 128.2813 - mae: 0.4070 - root_mean_squared_error: 11.3261 - mean_squared_logarithmic_error: 0.0793
Epoch 21/100
23/23 [==============================] - 0s 788us/step - loss: 116.9986 - mse: 116.9986 - mae: 0.3808 - root_mean_squared_error: 10.8166 - mean_squared_logarithmic_error: 0.0761
Epoch 22/100
23/23 [==============================] - 0s 816us/step - loss: 105.3287 - mse: 105.3287 - mae: 0.3597 - root_mean_squared_error: 10.2630 - mean_squared_logarithmic_error: 0.0743
Epoch 23/100
23/23 [==============================] - 0s 8

23/23 [==============================] - 0s 907us/step - loss: 1674.2576 - mse: 1674.2576 - mae: 10.3645 - root_mean_squared_error: 40.9177 - mean_squared_logarithmic_error: 4.2667
Epoch 3/100
23/23 [==============================] - 0s 788us/step - loss: 1440.2778 - mse: 1440.2778 - mae: 8.4880 - root_mean_squared_error: 37.9510 - mean_squared_logarithmic_error: 3.4776
Epoch 4/100
23/23 [==============================] - 0s 773us/step - loss: 1273.1382 - mse: 1273.1382 - mae: 6.6798 - root_mean_squared_error: 35.6811 - mean_squared_logarithmic_error: 2.6588
Epoch 5/100
23/23 [==============================] - 0s 920us/step - loss: 1085.9259 - mse: 1085.9259 - mae: 5.2023 - root_mean_squared_error: 32.9534 - mean_squared_logarithmic_error: 1.9693
Epoch 6/100
23/23 [==============================] - 0s 993us/step - loss: 964.4950 - mse: 964.4950 - mae: 4.0523 - root_mean_squared_error: 31.0563 - mean_squared_logarithmic_error: 1.4267
Epoch 7/100
23/23 [==============================] - 

23/23 [==============================] - 0s 1ms/step - loss: 0.2870 - mse: 0.2870 - mae: 0.0984 - root_mean_squared_error: 0.5357 - mean_squared_logarithmic_error: 0.0534
Epoch 90/100
23/23 [==============================] - 0s 895us/step - loss: 0.2852 - mse: 0.2852 - mae: 0.0981 - root_mean_squared_error: 0.5341 - mean_squared_logarithmic_error: 0.0533
Epoch 91/100
23/23 [==============================] - 0s 997us/step - loss: 0.2840 - mse: 0.2840 - mae: 0.0980 - root_mean_squared_error: 0.5329 - mean_squared_logarithmic_error: 0.0533
Epoch 92/100
23/23 [==============================] - 0s 906us/step - loss: 0.2824 - mse: 0.2824 - mae: 0.0978 - root_mean_squared_error: 0.5314 - mean_squared_logarithmic_error: 0.0532
Epoch 93/100
23/23 [==============================] - 0s 930us/step - loss: 0.2814 - mse: 0.2814 - mae: 0.0976 - root_mean_squared_error: 0.5305 - mean_squared_logarithmic_error: 0.0532
Epoch 94/100
23/23 [==============================] - 0s 2ms/step - loss: 0.2800 - ms

Epoch 30/100
23/23 [==============================] - 0s 1ms/step - loss: 106.8764 - mse: 106.8764 - mae: 0.4640 - root_mean_squared_error: 10.3381 - mean_squared_logarithmic_error: 0.1297
Epoch 31/100
23/23 [==============================] - 0s 965us/step - loss: 99.5781 - mse: 99.5781 - mae: 0.4439 - root_mean_squared_error: 9.9789 - mean_squared_logarithmic_error: 0.1242
Epoch 32/100
23/23 [==============================] - 0s 1ms/step - loss: 94.8790 - mse: 94.8790 - mae: 0.4219 - root_mean_squared_error: 9.7406 - mean_squared_logarithmic_error: 0.1161
Epoch 33/100
23/23 [==============================] - 0s 1ms/step - loss: 87.2138 - mse: 87.2138 - mae: 0.4036 - root_mean_squared_error: 9.3388 - mean_squared_logarithmic_error: 0.1112
Epoch 34/100
23/23 [==============================] - 0s 1ms/step - loss: 81.5576 - mse: 81.5576 - mae: 0.3876 - root_mean_squared_error: 9.0309 - mean_squared_logarithmic_error: 0.1068
Epoch 35/100
23/23 [==============================] - 0s 1ms/step

In [17]:
%tensorboard --logdir '/home/bavanya/Documents/Training_curves/cnn_propv347/'

In [18]:
total_training_time

[3.1847496032714844,
 2.978210210800171,
 3.1017980575561523,
 3.0895497798919678,
 3.0374343395233154,
 3.099594831466675,
 3.0459604263305664,
 3.2767980098724365,
 3.0942094326019287,
 3.1048877239227295,
 3.029768228530884,
 3.0376968383789062,
 3.1282548904418945,
 3.0257437229156494,
 3.000262498855591,
 3.021921157836914,
 3.062716484069824,
 3.032257556915283,
 3.045494318008423,
 3.170402765274048,
 3.1917173862457275,
 3.1443161964416504,
 3.193380117416382,
 3.2090227603912354,
 2.976142406463623,
 2.8823742866516113,
 2.888697385787964,
 2.7001700401306152,
 2.743835687637329,
 3.1195151805877686]

In [19]:
print("avg training time: " + str(sum(total_training_time)/30))

avg training time: 3.0538960774739583


In [20]:
total_training_loss_history

[{'loss': [1279.0428466796875,
   1128.4388427734375,
   1007.43359375,
   891.7160034179688,
   800.4310913085938,
   709.250732421875,
   634.3892211914062,
   575.8219604492188,
   511.4364013671875,
   456.0167236328125,
   412.2619934082031,
   372.9451904296875,
   334.04864501953125,
   305.4030456542969,
   273.70367431640625,
   252.7452392578125,
   227.16563415527344,
   207.97860717773438,
   192.3953857421875,
   175.8349151611328,
   161.76914978027344,
   149.60264587402344,
   138.9354248046875,
   127.8030776977539,
   119.31715393066406,
   111.21356201171875,
   103.7629165649414,
   96.83171081542969,
   90.76477813720703,
   86.41093444824219,
   80.3566665649414,
   75.52447509765625,
   71.69912719726562,
   67.93274688720703,
   64.09783935546875,
   60.596656799316406,
   57.82500076293945,
   54.709686279296875,
   52.09096145629883,
   50.58945083618164,
   47.23374938964844,
   45.874271392822266,
   42.98569869995117,
   41.41141891479492,
   39.29821014404

In [21]:
sum_mse = 0
sum_mae = 0
training_mse = []
training_mae = []
for i in range(30):
    sum_mse += float(total_training_loss_history[i]['mse'][-1])
    sum_mae += float(total_training_loss_history[i]['mae'][-1])
    training_mse.append(total_training_loss_history[i]['mse'][-1])
    training_mae.append(total_training_loss_history[i]['mae'][-1])
print("avg mse of training: " + str(sum_mse/30))
print("avg mae of training: " + str(sum_mae/30))

avg mse of training: 2.1262530306975047
avg mae of training: 0.11409310897191366


In [22]:
total_scores

[[0.24869565665721893,
  0.24869565665721893,
  0.08869565278291702,
  0.49869394302368164,
  0.049235641956329346],
 [0.24869565665721893,
  0.24869565665721893,
  0.08869565278291702,
  0.49869394302368164,
  0.049235641956329346],
 [0.4537064731121063,
  0.4537064731121063,
  0.10757793486118317,
  0.6735773682594299,
  0.059871215373277664],
 [0.2596709728240967,
  0.2596709728240967,
  0.09536901116371155,
  0.5095792412757874,
  0.052686430513858795],
 [0.24915733933448792,
  0.24915733933448792,
  0.08983481675386429,
  0.4991566240787506,
  0.04955140873789787],
 [0.5805825591087341,
  0.5805825591087341,
  0.11272051930427551,
  0.7619596719741821,
  0.06187254562973976],
 [0.24869565665721893,
  0.24869565665721893,
  0.08869565278291702,
  0.49869394302368164,
  0.049235641956329346],
 [0.24869565665721893,
  0.24869565665721893,
  0.08869565278291702,
  0.49869394302368164,
  0.049235641956329346],
 [0.24869565665721893,
  0.24869565665721893,
  0.08869565278291702,
  0.498

In [1]:
a = [[0.24869565665721893,
  0.24869565665721893,
  0.08869565278291702,
  0.49869394302368164,
  0.049235641956329346],
 [0.24869565665721893,
  0.24869565665721893,
  0.08869565278291702,
  0.49869394302368164,
  0.049235641956329346],
 [0.4537064731121063,
  0.4537064731121063,
  0.10757793486118317,
  0.6735773682594299,
  0.059871215373277664],
 [0.2596709728240967,
  0.2596709728240967,
  0.09536901116371155,
  0.5095792412757874,
  0.052686430513858795],
 [0.24915733933448792,
  0.24915733933448792,
  0.08983481675386429,
  0.4991566240787506,
  0.04955140873789787],
 [0.5805825591087341,
  0.5805825591087341,
  0.11272051930427551,
  0.7619596719741821,
  0.06187254562973976],
 [0.24869565665721893,
  0.24869565665721893,
  0.08869565278291702,
  0.49869394302368164,
  0.049235641956329346],
 [0.24869565665721893,
  0.24869565665721893,
  0.08869565278291702,
  0.49869394302368164,
  0.049235641956329346],
 [0.24869565665721893,
  0.24869565665721893,
  0.08869565278291702,
  0.49869394302368164,
  0.049235641956329346],
 [0.24869565665721893,
  0.24869565665721893,
  0.08869565278291702,
  0.49869394302368164,
  0.049235641956329346],
 [0.24869565665721893,
  0.24869565665721893,
  0.08869565278291702,
  0.49869394302368164,
  0.049235641956329346],
 [0.5234100222587585,
  0.5234100222587585,
  0.1304067075252533,
  0.7234708070755005,
  0.07476818561553955],
 [0.24869565665721893,
  0.24869565665721893,
  0.08869565278291702,
  0.49869394302368164,
  0.049235641956329346],
 [0.24869565665721893,
  0.24869565665721893,
  0.08869565278291702,
  0.49869394302368164,
  0.049235641956329346],
 [0.2676374614238739,
  0.2676374614238739,
  0.14820794761180878,
  0.517336905002594,
  0.0599919930100441],
 [0.34830328822135925,
  0.34830328822135925,
  0.10187846422195435,
  0.5901722311973572,
  0.057260312139987946],
 [0.24869565665721893,
  0.24869565665721893,
  0.08869565278291702,
  0.49869394302368164,
  0.049235641956329346],
 [0.25856485962867737,
  0.25856485962867737,
  0.10847608745098114,
  0.5084927082061768,
  0.05505848675966263],
 [0.24869565665721893,
  0.24869565665721893,
  0.08869565278291702,
  0.49869394302368164,
  0.049235641956329346],
 [0.2795996069908142,
  0.2795996069908142,
  0.09602682292461395,
  0.5287718176841736,
  0.053979724645614624],
 [3.708902359008789,
  3.708902359008789,
  0.24451901018619537,
  1.9258511066436768,
  0.12117443233728409],
 [0.24869565665721893,
  0.24869565665721893,
  0.08869565278291702,
  0.49869394302368164,
  0.049235641956329346],
 [0.31463849544525146,
  0.31463849544525146,
  0.10086844861507416,
  0.5609264373779297,
  0.05640260875225067],
 [0.2612345516681671,
  0.2612345516681671,
  0.09336542338132858,
  0.5111110806465149,
  0.0521942600607872],
 [0.24869579076766968,
  0.24869579076766968,
  0.08871087431907654,
  0.4986940920352936,
  0.04923577234148979],
 [0.24869565665721893,
  0.24869565665721893,
  0.08869565278291702,
  0.49869394302368164,
  0.049235641956329346],
 [0.2563462555408478,
  0.2563462555408478,
  0.17818939685821533,
  0.5063064694404602,
  0.05822794511914253],
 [0.24869565665721893,
  0.24869565665721893,
  0.08869565278291702,
  0.49869394302368164,
  0.049235641956329346],
 [0.24916328489780426,
  0.24916328489780426,
  0.08959747105836868,
  0.49916258454322815,
  0.04953915253281593],
 [10.939496040344238,
  10.939496040344238,
  0.27872398495674133,
  3.307490825653076,
  0.10510328412055969]]

In [2]:
test_mae = []
for i in range(10):
    test_mae.append(a[i][2])
sum(test_mae)/10


In [3]:
test_mae

[0.08869565278291702,
 0.08869565278291702,
 0.10757793486118317,
 0.09536901116371155,
 0.08983481675386429,
 0.11272051930427551,
 0.08869565278291702,
 0.08869565278291702,
 0.08869565278291702,
 0.08869565278291702,
 0.08869565278291702,
 0.1304067075252533,
 0.08869565278291702,
 0.08869565278291702,
 0.14820794761180878,
 0.10187846422195435,
 0.08869565278291702,
 0.10847608745098114,
 0.08869565278291702,
 0.09602682292461395,
 0.24451901018619537,
 0.08869565278291702,
 0.10086844861507416,
 0.09336542338132858,
 0.08871087431907654,
 0.08869565278291702,
 0.17818939685821533,
 0.08869565278291702,
 0.08959747105836868,
 0.27872398495674133]

In [26]:
sum_mse = 0
sum_mae = 0
sum_are = 0
test_mse = []
test_mae = []
for i in range(30):
    sum_mse += float(total_scores[i][0])
    sum_mae += float(total_scores[i][2])
    test_mse.append(total_scores[i][0])
    test_mae.append(total_scores[i][2])
    sum_are += float(test_ARE[i])
print("avg mse: " + str(sum_mse/30))
print("avg mae: " + str(sum_mae/30))
print("avg are: " + str(sum_are/30))

avg mse: 0.7560282851258914
avg mae: 0.11020706867178282
avg are: 0.05565217509865761


In [27]:
print("median training mse: " + str(statistics.median(training_mse)))
print("median testing mse: " + str(statistics.median(test_mse)))

median training mse: 0.3318815231323242
median testing mse: 0.2489265650510788


In [28]:
print("median training mae: " + str(statistics.median(training_mae)))
print("median testing mae: " + str(statistics.median(test_mae)))

median training mae: 0.0956219844520092
median testing mae: 0.08915417268872261


In [29]:
print("median testing ARE: " + str(statistics.median(test_ARE)))

median testing ARE: 0.05565217509865761


In [30]:
test_ARE

[0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175,
 0.055652175]